In [ ]:
#trades information
from binance import Client

%run ./Binance_Keys.ipynb

client = Client(api_key, api_secret)

import nest_asyncio
nest_asyncio.apply()

import datetime
import time
import asyncio
from binance import AsyncClient, BinanceSocketManager


async def main():
    client = await AsyncClient.create()
    bm = BinanceSocketManager(client)
    # start any sockets here, i.e a trade socket
    ts = bm.trade_socket('BTCUSDT')
    # then start receiving messages
    async with ts as tscm:
        while True:
            res = await tscm.recv()
            event = res['e']
            date_ms = res['E']
            date1 = datetime.datetime.fromtimestamp(date_ms / 1000).strftime("%Y-%m-%d %H:%M:%S")
            symbol = res['s']
            price = res['p']
            quant = res['q']
            print('Operação: '+event,"Data: " +date1,'Symbol: ' +symbol,'Preço: ' +price,"Quantidade: "+ quant)
            time.sleep(1)
            
            

    await client.close_connection()

if __name__ == "__main__":

    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())

In [ ]:
#ALL informations
from binance import Client

%run ./Binance_Keys.ipynb

import datetime
import time
import nest_asyncio
nest_asyncio.apply()

import asyncio
from binance import AsyncClient

async def main():
    client = await AsyncClient.create(api_key, api_secret)
    
    while True:
        res = await client.get_symbol_info('BTCUSDT')
        time.sleep(1)
        print(res)

    await client.close_connection()

if __name__ == "__main__":

    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())

In [ ]:
#Candles information
import websocket, json
import pandas as pd


crypto = 'btcusdt'
interval = '1m'
socket = f'wss://stream.binance.com:9443/ws/{crypto}@kline_{interval}'

def on_message(ws, message):
    print(message)

def on_close(ws, close_status_code, close_msg):
    print("Connection Closed")

ws = websocket.WebSocketApp(socket,on_message=on_message, on_close=on_close)

closes, highs, lows, volume, Times, Open = [], [], [], [], [], []

def on_message(ws, message):
    json_message = json.loads(message)
    candle = json_message['k']
    is_candle_closed = candle['x']
    close = candle['c']
    high = candle['h']
    low = candle['l']
    vol = candle['v']
    time = candle['T']
    opens = candle['o']
    
    if is_candle_closed:
        closes.append(float(close))
        Open.append(float(opens))
        highs.append(float(high))
        lows.append(float(low))
        volume.append(float(vol))
        Times.append(float(time))
        df = pd.DataFrame(list(zip(Open, highs, lows, volume, Times, closes)), columns = ['Open','High','Low','Volume','Time','Close'])
        df = df.set_index('Time')
        df.index = pd.to_datetime(df.index, unit = 'ms')
        df.index = df.index.strftime("%Y-%m-%d %H:%M:%S")
        print(df)     
        
    
    
def on_close(ws, close_status_code, close_msg):
    print("Connection Closed")

ws = websocket.WebSocketApp(socket,on_message=on_message, on_close=on_close)

ws.run_forever()